<a href="https://colab.research.google.com/github/adriannag9/machine-learning/blob/main/ru_sentiment_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 1. Import bibliotek
import pandas as pd
import numpy as np
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, preprocessing, linear_model, metrics, svm
from sklearn import ensemble

# Biblioteki do lematyzacji języka rosyjskiego
import pymorphy2
from pymystem3 import Mystem

# Pobieranie zasobów NLTK
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('omw-1.4')
nltk.download('wordnet')

# 2. Import dataset
# Ścieżka do Twojego pliku o rozmiarze 177 MB
raw_data = pd.read_csv('/content/drive/My Drive/projekt_ML/sentiment_dataset.csv')

# 3. Wypełnienie list (Odwzorowanie logiki 1:1 z oryginału)
texts = raw_data['text'].astype(str).tolist()
labels = raw_data['label'].tolist()

# 4. Utworzenie DataFrame
trainDF = pd.DataFrame()
trainDF['text'] = texts
trainDF['label'] = labels

# 5. Wyświetlenie rekordów i statystyk (Dla 290 458 rekordów)
print(trainDF.head(100))
print(trainDF.describe())

In [ ]:
# --- ANALIZA DANYCH ---
# 1. Sprawdzenie liczby słów w każdej recenzji
train = trainDF.copy()
train['word_count'] = train['text'].apply(lambda x: len(str(x).split(" ")))

print("Podgląd liczby słów:")
print(train[['text','word_count']].head())

In [ ]:
# Statystyki dla liczby słów (odpowiednik printów z oryginału)
print("\nStatystyki liczby słów:")
print(train['word_count'].max())
print(train['word_count'].min())
print(train['word_count'].mean())


Statystyki liczby słów:
463
1
51.404419916132454


In [ ]:
# 2. Sprawdzenie liczby znaków (łącznie ze spacjami)
train['char_count'] = train['text'].str.len()
print("\nPodgląd liczby znaków:")
print(train[['text','char_count']].head())


Podgląd liczby znaków:
                                                text  char_count
0  Пальто красивое, но пришло с дырой в молнии. П...         337
1  Очень долго шел заказ,ждала к новому году,приш...          82
2  Могу сказать одно, брюки нормальные, НО они бы...         284
3  Доставка быстрая, меньше месяца. Заказывали ра...         128
4  Мне не очень  понравилось это платье. Размер  ...         120


In [ ]:
# 3. Średnia długość słowa
def avg_word(sentence):
  words = str(sentence).split()
  if len(words) == 0:
    return 0
  return (sum(len(word) for word in words)/len(words))

train['avg_word'] = train['text'].apply(lambda x: avg_word(x))
print("\nŚrednia długość słowa:")
print(train[['text','avg_word']].head())


Średnia długość słowa:
                                                text  avg_word
0  Пальто красивое, но пришло с дырой в молнии. П...  5.607843
1  Очень долго шел заказ,ждала к новому году,приш...  6.545455
2  Могу сказать одно, брюки нормальные, НО они бы...  5.130435
3  Доставка быстрая, меньше месяца. Заказывали ра...  5.736842
4  Мне не очень  понравилось это платье. Размер  ...  5.555556


In [ ]:
# 1. Liczenie stop-words (słów nieistotnych) - dla rosyjskiego!
stop = stopwords.words('russian')
train['stopwords'] = train['text'].apply(lambda x: len([x for x in str(x).split() if x in stop]))
print("Statystyki stop-words (Top 5):")
print(train[['text','stopwords']].head())

Statystyki stop-words (Top 5):
                                                text  stopwords
0  Пальто красивое, но пришло с дырой в молнии. П...         11
1  Очень долго шел заказ,ждала к новому году,приш...          4
2  Могу сказать одно, брюки нормальные, НО они бы...         14
3  Доставка быстрая, меньше месяца. Заказывали ра...          4
4  Мне не очень  понравилось это платье. Размер  ...          3


In [ ]:
# 2. Liczenie liczb w tekście
train['numerics'] = train['text'].apply(lambda x: len([x for x in str(x).split() if x.isnumeric()]))
print("\nStatystyki liczb (Top 5):")
print(train[['text','numerics']].head())


Statystyki liczb (Top 5):
                                                text  numerics
0  Пальто красивое, но пришло с дырой в молнии. П...         0
1  Очень долго шел заказ,ждала к новому году,приш...         0
2  Могу сказать одно, брюки нормальные, НО они бы...         0
3  Доставка быстрая, меньше месяца. Заказывали ра...         0
4  Мне не очень  понравилось это платье. Размер  ...         0


In [ ]:
# --- CZYSZCZENIE (Preprocessing) ---

# Zachowujemy wersję przed czyszczeniem (tak jak w oryginale)
trainDFRaw = trainDF.copy()

# A. Zamiana na małe litery
trainDF['text'] = trainDF['text'].apply(lambda x: " ".join(str(x).lower() for x in str(x).split()))

# B. Usuwanie znaków specjalnych i interpunkcji
# Używamy regex, który zachowuje litery (w tym cyrylicę) i spacje
trainDF['text'] = trainDF['text'].str.replace(r'[^\w\s]', '', regex=True)

# C. Usuwanie rosyjskich stop-words
trainDF['text'] = trainDF['text'].apply(lambda x: " ".join(x for x in str(x).split() if x not in stop))

print("\nTekst po wstępnym czyszczeniu (małe litery, brak interpunkcji i stop-words):")
print(trainDF['text'].head())


Tekst po wstępnym czyszczeniu (małe litery, brak interpunkcji i stop-words):
0    пальто красивое пришло дырой молнии просила вы...
1    очень долго шел заказждала новому годупришел н...
2    могу сказать одно брюки нормальные порваны мал...
3    доставка быстрая меньше месяца заказывали разм...
4    очень понравилось это платье размер l подошёл ...
Name: text, dtype: object


In [ ]:
# --- USUNIĘCIE SŁÓW CZĘSTYCH I RZADKICH ORAZ LEMATYZACJA ---

# 1. Przygotowanie listy 20 najczęstszych słów (Top 20 Frequent Words)
freq = pd.Series(' '.join(trainDF['text']).split()).value_counts()[:20]
print("Najczęstsze słowa do usunięcia:")
print(freq)

Najczęstsze słowa do usunięcia:
это           115810
очень         109010
просто         41663
аниме          35086
фильм          31993
всё            29624
10             25739
вообще         22314
размер         20762
хотя           19585
время          18904
ещё            18072
сюжет          17975
деньги         17693
качество       17178
2              16456
смотреть       15528
товар          15091
рекомендую     14750
которые        14686
Name: count, dtype: int64


In [ ]:
# Usuwanie najczęstszych słów
freq_list = list(freq.index)
trainDF['text'] = trainDF['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_list))

In [ ]:
# 2. Przygotowanie listy 20 najrzadszych słów (Top 20 Rare Words)
# rare = pd.Series(' '.join(trainDF['text']).split()).value_counts()[-20:]
# print("\nNajrzadsze słowa do usunięcia:")
# print(rare) - to podejście się nie sprawdza, jest to 20 losowych słów, przy tak dużym zbiorze jest więcej słów, które występują tylko raz. HAPAX LEGOMENA
# Usuwanie najrzadszych słów
# rare_list = list(rare.index)
# trainDF['text'] = trainDF['text'].apply(lambda x: " ".join(x for x in x.split() if x not in rare_list))

In [ ]:
# 1. Logika lingwistyczna: Usuwanie wszystkich słów występujących tylko RAZ
all_words = pd.Series(' '.join(trainDF['text']).split())
word_counts = all_words.value_counts()
rare_words_logical = word_counts[word_counts == 1].index

print(f"Liczba unikalnych słów do usunięcia (występujących tylko raz): {len(rare_words_logical)}")

Liczba unikalnych słów do usunięcia (występujących tylko raz): 814773


In [ ]:
# Usuwamy je
trainDF['text'] = trainDF['text'].apply(lambda x: " ".join(x for x in x.split() if x not in rare_words_logical))

In [ ]:
import inspect
from collections import namedtuple

# Definiujemy strukturę, którą zrozumie stary kod pymorphy2
ArgSpec = namedtuple('ArgSpec', ['args', 'varargs', 'keywords', 'defaults'])

def getargspec_patch(func):
    args, varargs, varkw, defaults, kwonlyargs, kwonlydefaults, annotations = inspect.getfullargspec(func)
    return ArgSpec(args, varargs, varkw, defaults)

# Podmieniamy funkcję w module inspect
inspect.getargspec = getargspec_patch

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

# Teraz ten kod powinien przejść bez błędu ValueError:
sample_text = "пальто пришло красивое"
lemmas = [morph.parse(word)[0].normal_form for word in sample_text.split()]
print(lemmas) # Powinno wyjść: ['пальто', 'прийти', 'красивый']
import inspect
# Naprawa błędu zgodności dla Python 3.11+ i pymorphy2
if not hasattr(inspect, 'getargspec'):
    inspect.getargspec = inspect.getfullargspec

# 2. PORÓWNANIE LEMATYZACJI (na próbce 5 pierwszych recenzji)
morph = pymorphy2.MorphAnalyzer()
mystem = Mystem()

sample_texts = trainDF['text'].head(5).tolist()

print("\n--- PORÓWNANIE LEMATYZACJI ---")
for i, text in enumerate(sample_texts):
    print(f"\nOryginał: {text[:100]}...")

    # Pymorphy2
    py_lemmas = [morph.parse(word)[0].normal_form for word in text.split()]
    print(f"Pymorphy2: {' '.join(py_lemmas)[:100]}...")

    # Mystem
    myst_lemmas = mystem.lemmatize(text)
    print(f"Mystem: {''.join(myst_lemmas).strip()[:100]}...")

# --- WYBÓR DO PROJEKTU: Mystem (wybrany ze względu na lepszą jakość w testach) ---
print("Uruchamiam wydajną lematyzację Mystem dla całego zbioru (290k rekordów)...")

# Metoda wydajnego przetwarzania dużych tekstów przez Mystem
def mystem_full_process(texts_list):
    # Łączymy wszystkie teksty specjalnym separatorem, aby Mystem przetworzył je jako jeden ciąg
    full_text = " |separator| ".join(texts_list)
    lemmatized = mystem.lemmatize(full_text)

    # Składamy z powrotem w listę oczyszczonych tekstów
    processed_text = "".join(lemmatized)
    return [t.strip() for t in processed_text.split("|separator|")]

# Przetwarzanie w paczkach po 5000 wierszy, aby nie przepełnić pamięci
all_texts = trainDF['text'].tolist()
final_lemmas = []
batch_size = 5000

for i in range(0, len(all_texts), batch_size):
    batch = all_texts[i:i + batch_size]
    final_lemmas.extend(mystem_full_process(batch))
    if (i // batch_size) % 5 == 0:
        print(f"Przetworzono {i} / {len(all_texts)} rekordów...")

trainDF['text'] = final_lemmas

print("\nPełna lematyzacja Mystem zakończona. Podgląd wyników:")
print(trainDF['text'].head())

In [ ]:
# 1. Podział na zbiór treningowy i walidacyjny
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'])

# 2. Kodowanie etykiet
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.transform(valid_y)

# 3. Wektoryzacja TF-IDF
# min_df=2 -> automatycznie ignoruje 814k rzadkich słów
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000, min_df=2)
tfidf_vect.fit(trainDF['text'])

xtrain_tfidf = tfidf_vect.transform(train_x)
xvalid_tfidf = tfidf_vect.transform(valid_x)

print("Przygotowanie danych zakończone.")

In [ ]:
# --- TRENOWANIE I EWALUACJA MODELI ---

# 1. Uniwersalna funkcja do trenowania (Odwzorowanie 1:1 z oryginału)
def train_model(classifier, feature_vector_train, label, feature_vector_valid):
    # Trenowanie modelu
    classifier.fit(feature_vector_train, label)

    # Generowanie przewidywań dla zbioru testowego
    predictions = classifier.predict(feature_vector_valid)

    # Obliczanie metryk (Precision, Recall, F1, Accuracy)
    precision = metrics.precision_score(predictions, valid_y, average='weighted')
    recall = metrics.recall_score(predictions, valid_y, average='weighted')
    f1 = metrics.f1_score(predictions, valid_y, average='weighted')
    accuracy = metrics.accuracy_score(predictions, valid_y)

    return [precision, recall, f1, accuracy]

# Słownik do przechowywania wyników dla porównania
accuracy_compare = {}

# MODEL 1 - Regresja Logistyczna (Logistic Regression)
results_lr = train_model(linear_model.LogisticRegression(max_iter=1000), xtrain_tfidf, train_y, xvalid_tfidf)
accuracy_compare['LR'] = results_lr
print ("LR, TF-IDF Accuracy: ", results_lr[3])

# MODEL 2 - Support Vector Machine (SVM)
# Uwaga: Na tak dużym zbiorze SVM może zająć kilka minut
results_svm = train_model(svm.LinearSVC(), xtrain_tfidf, train_y, xvalid_tfidf)
accuracy_compare['SVM'] = results_svm
print ("SVM, TF-IDF Accuracy: ", results_svm[3])

# MODEL 3 - Random Forest (Las Losowy)
results_rf = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf, train_y, xvalid_tfidf)
accuracy_compare['RF'] = results_rf
print ("RF, TF-IDF Accuracy: ", results_rf[3])

LR, TF-IDF Accuracy:  0.6834813743716863


C:\Users\PlumResearch\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


SVM, TF-IDF Accuracy:  0.6801625008607037


KeyboardInterrupt: 

In [ ]:
# --- ZAPISYWANIE MODELI DO PLIKÓW ---
import joblib
import os

# Tworzymy folder na modele, jeśli nie istnieje
if not os.path.exists('models'):
    os.makedirs('models')

# Zapisujemy każdy model do oddzielnego pliku
joblib.dump(accuracy_compare, 'models/results_dict.pkl')
joblib.dump(tfidf_vect, 'models/tfidf_vectorizer.pkl')

# Jeśli modele skończyły się trenować, zapisujemy je (pamiętaj, że RF może jeszcze trwać)
if 'LR' in accuracy_compare:
    # Zapisujemy same obiekty modeli (można je później wczytać do predykcji)
    # Uwaga: poniższe zmienne muszą istnieć (musisz mieć skończony trening)
    # joblib.dump(classifier_lr, 'models/model_lr.pkl') # jeśli zapisałaś je w zmiennych

    print("Wyniki i wektoryzator zostały zapisane w folderze 'models'.")
    print("Teraz nawet po zamknięciu laptopa, odtworzysz je w sekundę.")